<a href="https://colab.research.google.com/github/ayan524-ai/Chatbot-for-FAQs/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd

# Load the dataset you just uploaded
df = pd.read_csv('faq.csv')

# Display the first 5 rows in a clean table
df.head()

,Question,Answer
0,What are the library hours?,"The library is open from 9 AM to 8 PM, Monday ..."
1,How can I renew a book?,You can renew a book online through your libra...
2,Can I borrow a laptop?,"Yes, laptops are available for a 4-hour loan p..."
3,How many books can I check out at once?,You can check out up to 10 books at a time.
4,Is there a fine for overdue books?,"Yes, the fine is $1 per day for each overdue b..."


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Get the list of English stop words
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if not word in stop_words]
    return " ".join(tokens)

# Apply the function to your 'Question' column
df['processed_question'] = df['Question'].apply(preprocess_text)

# Display the original vs. processed questions
df.head()

,Question,Answer,processed_question
0,What are the library hours?,"The library is open from 9 AM to 8 PM, Monday ...",library hours
1,How can I renew a book?,You can renew a book online through your libra...,renew book
2,Can I borrow a laptop?,"Yes, laptops are available for a 4-hour loan p...",borrow laptop
3,How many books can I check out at once?,You can check out up to 10 books at a time.,many books check
4,Is there a fine for overdue books?,"Yes, the fine is $1 per day for each overdue b...",fine overdue books


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Create the TF-IDF matrix from the processed questions
tfidf_matrix = vectorizer.fit_transform(df['processed_question'])

print("Text has been successfully vectorized.")
print("Shape of the TF-IDF matrix:", tfidf_matrix.shape)

Text has been successfully vectorized.
Shape of the TF-IDF matrix: (10, 21)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_response(user_query):
    # 1. Preprocess the user's query
    processed_query = preprocess_text(user_query)

    # 2. Vectorize the user's query
    query_vector = vectorizer.transform([processed_query])

    # 3. Calculate cosine similarity between the user's query and all FAQ questions
    similarities = cosine_similarity(query_vector, tfidf_matrix)

    # 4. Find the index of the most similar question
    most_similar_index = np.argmax(similarities)

    # 5. Return the corresponding answer
    return df['Answer'].iloc[most_similar_index]

In [ ]:
print("Chatbot: Hello! I'm here to help. Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    response = get_response(user_input)
    print("Chatbot:", response)

Chatbot: Hello! I'm here to help. Type 'quit' to exit.
You: what time do you open?
Chatbot: The library is open from 9 AM to 8 PM, Monday to Saturday.
You: how to renew books?
Chatbot: You can renew a book online through your library account or in person at the front desk.
You: hello
Chatbot: The library is open from 9 AM to 8 PM, Monday to Saturday.
You: now what
Chatbot: The library is open from 9 AM to 8 PM, Monday to Saturday.


KeyboardInterrupt: Interrupted by user